# Code - Preprocessing and Transformation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from collections import Counter

# pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('steam_games.csv')

def wrapErrorDelete(path):
    try:
        os.remove(path)
    except:
        pass
        
def p(x):
    print(x)

In [3]:
df = df[df['name'].notna()]
df.drop(["discount_price", "publisher", "developer", "release_date", "desc_snippet", "types", "achievements", "recent_reviews", "mature_content"], axis=1, inplace=True)

In [4]:
for row in df.iterrows():
    if pd.isna(row[1][3]) and pd.isna(row[1][6]):
        print("deleting: ", row[1][1])
        df.drop(row[0], inplace=True)    

deleting:  Ghost In The Shell: Stand Alone Complex 2nd Gig
deleting:  Ghost In The Shell: Stand Alone Complex Season 1
deleting:  Gintama Season 4
deleting:  CRYENGINE - Sample Assets
deleting:  Worms Reloaded: The "Pre-order Forts and Hats" DLC Pack
deleting:  Krater - Soundtrack
deleting:  Dustforce Soundtrack
deleting:  The Incredible Adventures of Van Helsing II - OST
deleting:  D4: SWERY's Choice Costume Set -4 Cups of Coffee-
deleting:  D4: SWERY's Choice Costume Set -4 Bottles of Tequila-
deleting:  Kemono Friends Season 1
deleting:  New Game Season 1
deleting:  Showa Genroku Rakugo Shinju Season 1
deleting:  Con Man Season 1
deleting:  DayZ Tools
deleting:  Forge of Gods: Ultimate Titan Bundle
deleting:  Killing Floor "London's Finest" Character Pack
deleting:  Dungeon Defenders Capture the Flag Pre-Alpha Pass (Free DLC)
deleting:  Digital Sound Factory: Session Drummer 3 Expansions
deleting:  Mob Psycho 100 Season 1
deleting:  Re:ZERO -Starting Life in Another World- Season 1


In [5]:
duplicated_items = df[df["name"].duplicated()]["name"].values.tolist()
games = {}
print(df.shape)
for row in df.iterrows():
    if row[1][1] in duplicated_items:
        if row[1][1] in games:
            games[row[1][1]].append(row[1].isna().sum())
        else:
            games[row[1][1]] = [row[1].isna().sum()]
for item in games:
    games[item].sort()
    count = 0
    for row in df.iterrows():
        if row[1][1] == item and (row[1].isnull().sum() == games[item][0]):
            df.drop(row[0], inplace=True)
            count += 1
            print("deleting: ", row[1][1])
        
        if count + 1 == len(games[item]):
            break
print(df.shape)
df.to_csv("steam_games_p1.csv", index=False)

(40675, 11)
deleting:  RUSH
deleting:  Project Highrise
deleting:  Batman: Arkham City - Game of the Year Edition
deleting:  Angels of Death
deleting:  Luna
deleting:  Flashback
deleting:  New York Bus Simulator
deleting:  Warhammer Quest
deleting:  City Builder
deleting:  Killing Time
deleting:  Jigsaw Puzzles
deleting:  Hide and Seek
deleting:  Hidden Object Adventure Bundle
deleting:  Arcade Bundle
deleting:  Arcade Bundle
deleting:  Soundtrack Edition
deleting:  Solitaire Collection
deleting:  Ultimate Arena
deleting:  Black Rose
deleting:  Solstice
deleting:  Surge
deleting:  Game & Soundtrack
deleting:  7 Days in Dream 光尘
deleting:  Bounce
deleting:  Game + Soundtrack
deleting:  Chaos Theory
deleting:  Deluxe Edition
deleting:  Deluxe Edition
deleting:  Rumpus
deleting:  Space Maze
deleting:  Spark
deleting:  The Mine
deleting:  Memories
deleting:  Ashes
deleting:  Dodge
deleting:  We Were Here Too: Supporter Edition
deleting:  The Deer
deleting:  Zombie Apocalypse
deleting:  Aft

In [6]:
df = pd.read_csv('steam_games_p1.csv')
all_review_cols = ["all_review_sentiment", "all_review_sentiment_num", "all_review_num"]
temp_df = []
idx = 0
for row in df.iterrows():
    if row[1][2] == None or pd.isna(row[1][2]):
        temp_df.append(["None", 0, 0])
    else:
        try:
            items = row[1][2].split('(')
            sentiment = items[0].lower().strip()[0:-1]
            items = items[1].split(')')
            total = int(items[0].strip().replace(',', ''))
            percent = int(items[1].split('%')[0].strip()[3:]) / 100
            temp_df.append([sentiment, int(total * percent), total])
        except:
            temp_df.append(["None", 0, 0])
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
all_review_df = pd.DataFrame(temp_df, columns=all_review_cols)
df.drop(["all_reviews"], axis=1, inplace=True)
df = pd.concat([df, all_review_df], axis=1)
#df['all_review_sentiment'] = df['all_review_sentiment'].astype('category').cat.codes
df['popular_tags'].fillna(df['genre'], inplace=True)
for i in range(len(df)):
    if pd.isna(df.loc[i, 'genre']):
        df.drop(i, inplace=True)
df.to_csv("steam_games_p2.csv", index=False)

12.271790341578328 %
24.541126815861798 %
36.810463290145265 %
49.07979976442874 %
61.34913623871221 %
73.61847271299568 %
85.88780918727915 %
98.15714566156262 %


In [7]:
df['all_review_sentiment'].astype('category').cat.categories

Index(['None', 'mixed', 'mostly negative', 'mostly positive', 'negative',
       'overwhelmingly negative', 'overwhelmingly positive', 'positive',
       'very negative', 'very positive'],
      dtype='object')

In [8]:
df = pd.read_csv('steam_games_p2.csv')

popular_tags = set()
pt_list = []

tag_list = df['popular_tags'].to_list()
genre_list = df['genre'].to_list()
for i in range(len(tag_list)):
    temp = tag_list[i].split(',')
    temp.extend(genre_list[i].split(','))
    temp = list(set(temp))
    pt_list.extend(temp)
    for j in temp:
        popular_tags.add(j.lower().strip())
pd.Series(pt_list).to_csv('popular_tags_freq.csv', index=False)

In [9]:
"""
popular_tags = list(popular_tags)
tag_df_cols = ["url"]
tag_df_cols.extend(popular_tags)

idx = 0
tag_map = {}
for tag in popular_tags:
    tag_map[tag] = idx + 2
    idx += 1"""
    
idx = 0
df = pd.read_csv('steam_games_p2.csv')
n = len(popular_tags)
temp_df = []

for row in df.iterrows():
    items = [row[1][0]]
    items.extend([0] * n)
    for tag in popular_tags:
        if row[1][2].lower().strip().find(tag) > -1:
            items[tag_map[tag]] = 1
        if row[1][5].lower().strip().find(tag) > -1:
            items[tag_map[tag]] = 1
    # tag_df.loc[idx] = items
    temp_df.append(items)
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
df.drop(["popular_tags", "genre"], axis=1, inplace=True)
tag_df = pd.DataFrame(temp_df, columns=tag_df_cols)
df_concat = pd.merge(df, tag_df, on='url', how='outer')
df_concat.to_csv("steam_games_tags.csv", index=False)

NameError: name 'tag_map' is not defined

In [ ]:
def format_csv(data):
    try:
        return list(set(data.split(',')))
    except:
        return []

df = pd.read_csv('steam_games_p2.csv')
df['total'] = df['popular_tags'] + df['genre'] + df['game_details']
df['total'] = df['total'].apply(format_csv)
df.drop(['popular_tags', 'genre', 'game_details', 'minimum_requirements', 'recommended_requirements', 'game_description', "languages", "all_review_sentiment_num", "all_review_num", "all_review_sentiment", "original_price"], axis=1, inplace=True)
df.to_csv('steam_games_genres.csv', index=False)

In [ ]:
df = pd.read_csv('steam_games_p2.csv')

In [ ]:
df["all_review_sentiment"].value_counts()

0    23133
1     4646
9     4491
7     3498
3     3290
2      775
6      316
4      135
8       37
5        7
Name: all_review_sentiment, dtype: int64

In [ ]:
df = pd.read_csv('steam_games_tags.csv')

gd_tags = set()
pt_list = []

tag_list = df['game_details'].to_list()
for i in range(len(tag_list)):
    try:
        temp = tag_list[i].split(',')
        temp = list(set(temp))
        pt_list.extend(temp)
        for j in temp:
            gd_tags.add(j.lower().strip())
    except:
        continue
pd.Series(pt_list).to_csv('game_details_freq.csv', index=False)

In [ ]:
gd_tags = list(gd_tags)
tag_df_cols = ["url"]
tag_df_cols.extend(gd_tags)

idx = 0
tag_map = {}
for tag in gd_tags:
    tag_map[tag] = idx + 2
    idx += 1
    
idx = 0
df = pd.read_csv('steam_games_tags.csv')
n = len(gd_tags)
temp_df = []
for row in df.iterrows():
    items = [row[1][0]]
    items.extend([0] * n)
    for tag in gd_tags:
        try:
            if row[1][2].lower().strip().find(tag) > -1:
                items[tag_map[tag]] = 5
        except:
            pass
    # tag_df.loc[idx] = items
    temp_df.append(items)
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
df.drop(["game_details"], axis=1, inplace=True)
tag_df = pd.DataFrame(temp_df, columns=tag_df_cols)
df_concat = pd.merge(df, tag_df, on='url', how='outer')
df_concat.to_csv("steam_games_tags_gd.csv", index=False)

In [ ]:
df = pd.read_csv('steam_games_tags_gd.csv')
language_tags = set()
pt_list = []

tag_list = df['languages'].to_list()
for i in range(len(tag_list)):
    try:
        temp = tag_list[i].split(',')
        temp = list(set(temp))
        pt_list.extend(temp)
        for j in temp:
            language_tags.add(j.lower().strip())
    except:
        continue
pd.Series(pt_list).to_csv('game_language_freq.csv', index=False)

In [ ]:
language_tags = list(language_tags)
tag_df_cols = ["url"]
tag_df_cols.extend(language_tags)

idx = 0
tag_map = {}
for tag in language_tags:
    tag_map[tag] = idx + 2
    idx += 1
    
idx = 0
df = pd.read_csv('steam_games_tags_gd.csv')
n = len(language_tags)
temp_df = []
for row in df.iterrows():
    items = [row[1][0]]
    items.extend([0] * n)
    for tag in language_tags:
        try:
            if row[1][2].lower().strip().find(tag) > -1:
                items[tag_map[tag]] = 1
        except:
            pass
    # tag_df.loc[idx] = items
    temp_df.append(items)
    idx += 1
    if idx % 5000 == 0:
        print(((idx + 1) / 40752) * 100, "%")
df.drop(["languages"], axis=1, inplace=True)
tag_df = pd.DataFrame(temp_df, columns=tag_df_cols)
df_concat = pd.merge(df, tag_df, on='url', how='outer')
df_concat.to_csv("steam_games_tags_languages.csv", index=False)

In [ ]:
df = pd.read_csv('steam_games_tags_gd.csv')

def format_price(price):
    price_str = str(price)
    if price_str[0] == "$":
        return float(price_str[1:])
    
    try:
        test = int(price_str[0])
        return float(price_str)
    except:
        return 0

df['original_price'] = df['original_price'].apply(format_price)
df.drop(['minimum_requirements', 'recommended_requirements', 'game_description', "languages", "all_review_sentiment_num", "all_review_num", "all_review_sentiment", "original_price"], axis=1, inplace=True)
df.to_csv('steam_games_final_format.csv', index=False)